# Steps for Preprocessing Aggregation Data

### 1. Calculate the predcited scores from the sentence classification model for every sentence in a case
> * Need a dataset woth colums of every case, every batch, every sentencece from every batch, the summaries from a case, and 0 / 1 labels relating to every sentence in every batch that says whether or not a sentence is in the summary.
* Need to upload the state dict from the trained model
* Just run the data through the trained model (Takes a very long time)

### 2. Calculate the positional weight for each batch and create a new column that mulitplies every score by the positional weight of its batch

### 3. Calculate the positional weight for each batch and create a new column that mulitplies every score by the positional weight of its batch
>* Need the state dict from the trained batch classification model. Calculated the predicted output for each batch.

Uploading the data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def save_as_pickle(filename, data):
  with open(filename, 'wb') as handle:   #Saving as a pickle file
    pickle.dump(data, handle)

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!pip install -q transformers  rouge-score sentence-transformers

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from tqdm.notebook import tqdm
import pickle
 
import plotly.express as px

import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
 
from transformers import AutoTokenizer, AutoModel 
nlp = spacy.load('en_core_web_lg')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
batched_df = pd.read_pickle('/content/gdrive/MyDrive/Thesis/Data/aggregation_data.pickle')

smaller_df = batched_df.head(300) #NEEDED TO CUT OFF NUMBER OF CASES ANALYZING FOR COMPLEXITY REASONS

state_dict = torch.load("/content/gdrive/MyDrive/Thesis/Models/state_dict.pt", map_location=torch.device('cpu'))

### 1. Calculate the predcited scores from the sentence classification model for every sentence in a case

In [ ]:
# get mean pooling for sentence bert models 
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class SentenceBertClass(torch.nn.Module):
    def __init__(self, model_name="bert-base-uncased", in_features=768):
        super(SentenceBertClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(model_name)
        self.pre_classifier = torch.nn.Linear(in_features*3, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)
        self.classifierSigmoid = torch.nn.Sigmoid()

    def forward(self, sent_ids, doc_ids, sent_mask, doc_mask):

        sent_output = self.l1(input_ids=sent_ids, attention_mask=sent_mask) 
        sentence_embeddings = mean_pooling(sent_output, sent_mask) 

        doc_output = self.l1(input_ids=doc_ids, attention_mask=doc_mask) 
        doc_embeddings = mean_pooling(doc_output, doc_mask)

        # elementwise product of sentence embs and doc embs
        combined_features = sentence_embeddings * doc_embeddings  

        # Concatenate input features and their elementwise product
        concat_features = torch.cat((sentence_embeddings, doc_embeddings, combined_features), dim=1)   
        
        pooler = self.pre_classifier(concat_features) 
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.classifierSigmoid(output) 

        return output

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

extractive_model = SentenceBertClass() 
extractive_model.load_state_dict(state_dict)
extractive_model.eval(); 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# tokenize text as required by BERT based models
def get_tokens(text, tokenizer):
  inputs = tokenizer.batch_encode_plus(
            text, 
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
  ids = inputs['input_ids']
  mask = inputs['attention_mask']
  return ids, mask

In [ ]:
# get predictions given some an array of sentences and their corresponding documents
def predict(model,sents, doc):
  sent_id, sent_mask = get_tokens(sents,tokenizer)
  sent_id, sent_mask = torch.tensor(sent_id, dtype=torch.long),torch.tensor(sent_mask, dtype=torch.long)
 
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = doc_id * len(sents), doc_mask* len(sents)
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  preds = model(sent_id, doc_id, sent_mask, doc_mask)
  return preds

In [ ]:
def batch_and_score(doc, model, doc_sentences, batch_size=3):
  '''
  All this function does is take in a document, a model, the list of sentences, it then batches the sentence and sends it though the predictor to generatre prediction scores
  '''
  scores = [] 

  # run predictions using some batch size
  for i in range(int(len(doc_sentences) / batch_size) + 1):
    batch_start = i*batch_size  
    batch_end = (i+1) * batch_size if (i+1) * batch_size < len(doc) else len(doc)-1
    batch = doc_sentences[batch_start: batch_end]
    if batch:
      preds = predict(model, batch, doc) 
      scores = scores + preds.tolist() 

  return scores

In [ ]:
def calc_sentence_score(batched_df, model):
  '''
  This is the function that actually claculates the predicted score for each sentence

  It results in a new column in the dataframe with the predcited score for each sentence
  '''

  all_pred_scores = []
  case_text = batched_df['case_text']
  summaries = batched_df['summaries']
  batches = batched_df['512_batches']
  batch_sentences = batched_df['batch_sentences']

  #Iterating though each case
  for i in tqdm(range(len(case_text))):
    if i % 10 == 0 and i != 0:
      print('Done with', i, 'cases, New File Saved')
      save_as_pickle('/content/gdrive/MyDrive/Thesis/Data/all_pred_scores.pickle', all_pred_scores)


    case_pred_outcomes = []
    relevant_batches = batches[i]
    relevant_batch_sentences = batch_sentences[i]

    #Iterating through each batch in the case
    for j in range(len(relevant_batches)):

      key_batch = relevant_batches[j]
      key_sents = relevant_batch_sentences[j]

      #Generating list of sents from each batch, list of predicted outcomes, list of whether each sentence is in the summmary, and whether each batch contains a summary
      scores = batch_and_score(key_batch, model, key_sents)

      case_pred_outcomes.append(scores) #Should be the scores for each batch in a list
  
    all_pred_scores.append(case_pred_outcomes)

  batched_df['pred_outcomes'] = all_pred_scores
  
  return batched_df

In [ ]:
import time
start_time = time.time()

scored_df = calc_sentence_score(smaller_df, extractive_model)

filename = '/content/gdrive/MyDrive/Thesis/Data/scored_df.pickle'
save_as_pickle(filename, scored_df)

end_time = time.time()
duration = end_time - start_time
print('Aggregation Preprocessing took:', duration)

In [ ]:
pred_scores = pd.read_pickle('/content/gdrive/MyDrive/Thesis/Data/all_pred_scores.pickle')

In [ ]:
print(scored_df)

### 2. Calculate the positional weight for each batch and create a new column that mulitplies every score by the positional weight of its batch

In [ ]:
import pickle
scored_df = pd.read_pickle('/content/gdrive/MyDrive/Thesis/Data/scored_df.pickle')

In [ ]:
def custom_gauss_score(pos, a = 0.03065278, b = 0.40486706, c = 0.29048349): #Got these variables from the data exploration code
  '''
  Write a function calculates the gauss weight based on position of the batch
  '''
  return a * np.exp(-(pos - b)**2 / (2 * c**2))

In [ ]:
def create_pos(inference_df):
  '''
  This function looks at each case and multiplies the outcomes in each batch by the gaussian result from the position of that batch within the case
  Need to nomralize the number of batches on a scale from 0 to 1 and calcualte the gauss
  '''
  all_pos_preds = []

  for index, row in inference_df.iterrows():

    batches = row['512_batches']
    pred_for_sentences = row['pred_outcomes']

    num_batches = len(batches)

    positions = np.linspace(0, 1, num_batches) #Calculating position of each batch based on number of batches

    updated_pos_preds = []
    for i in range(num_batches):

      p = positions[i] #The position of the batch
      preds_for_batch_sents = pred_for_sentences[i] #list of sentence scores within a batch
      gauss_score = custom_gauss_score(p)

      updated_values = [elem[0] * gauss_score for elem in preds_for_batch_sents] #Multiplying each element in a batch by its weight

      updated_pos_preds.append(updated_values)

    all_pos_preds.append(updated_pos_preds)

  inference_df['pos_update'] = all_pos_preds

  return inference_df


In [ ]:
scored_df = create_pos(scored_df)
save_as_pickle('/content/gdrive/MyDrive/Thesis/Data/inference_df.pickle', scored_df)

print(scored_df.columns)


In [ ]:
print(scored_df['pos_update'])

### 3. Calculate the positional weight for each batch and create a new column that mulitplies every score by the positional weight of its batch

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


class BatchBertClass(torch.nn.Module):
    def __init__(self, model_name="bert-base-uncased", in_features=768):
        super(BatchBertClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(model_name)
        self.pre_classifier = torch.nn.Linear(in_features, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)
        self.classifierSigmoid = torch.nn.Sigmoid()

    def forward(self, doc_ids, doc_mask):

        doc_output = self.l1(input_ids=doc_ids, attention_mask=doc_mask) 
        doc_embeddings = mean_pooling(doc_output, doc_mask)
        
        pooler = self.pre_classifier(doc_embeddings) 
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.classifierSigmoid(output) 

        return output

In [ ]:
batches_state_dict = torch.load("/content/gdrive/MyDrive/Thesis/Models/batch_label_state_dict.pt")

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

batch_class_model = BatchBertClass()
batch_class_model.load_state_dict(batches_state_dict)
batch_class_model.eval();

In [ ]:
def predict_batch(model, doc):
  '''
  This function makes the actual predictions
  '''
 
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  pred = model(doc_id, doc_mask)
  return pred

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
def create_bin_class(inference_df):
  '''
  This function looks at each case and multiplies the outcomes in each batch by the gaussian result from the position of that batch within the case
  Need to nomralize the number of batches on a scale from 0 to 1 and calcualte the gauss
  '''
  all_pos_preds = []

  for index, row in tqdm(inference_df.iterrows(), total=len(inference_df)):

    batches = row['512_batches']
    pred_for_sentences = row['pred_outcomes']

    updated_pos_preds = []
    for i in range(len(batches)):

      batch = batches[i]

      preds_for_batch_sents = pred_for_sentences[i] #list of sentence scores within a batch
      batch_score =  predict_batch(batch_class_model, batch) #Fill in here with
      batch_score = batch_score.tolist()[0][0]

      updated_values = [elem[0] * batch_score for elem in preds_for_batch_sents] #Multiplying each element in a batch by its weight

      updated_pos_preds.append(updated_values)

    all_pos_preds.append(updated_pos_preds)

  inference_df['bin_class_update'] = all_pos_preds

  return inference_df

In [ ]:
scored_df = create_bin_class(scored_df)
print(scored_df.columns)
save_as_pickle('/content/gdrive/MyDrive/Thesis/Data/scored_df.pickle', scored_df)

In [ ]:
print(scored_df['bin_class_update'])